You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Aggregate Operation:
o	Perform a simple "aggregate" operation on the retrieved launch data.
o	Aggregation Logic: Calculate the total number of launches for each year.
o	Return a print with the number of aggregate launches for each year
3.	Control Parameters and Debugging:
o	Include a variable at the beginning of the script to define the API endpoint URL, making it easily modifiable: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, starting aggregation, aggregation complete, number of years aggregated, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + aggregation). Print this time after the aggregation operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting aggregated summary dictionary (year counts) from the aggregate operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting aggregated JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
import requests
import logging
import time
import json
from datetime import datetime

# Configuration
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
HTTPBIN_POST_URL = "https://httpbin.org/post"

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def retrieve_launch_data():
    """Retrieve launch data from SpaceX API"""
    logger.info("Starting data retrieval from SpaceX API")
    
    try:
        response = requests.get(API_ENDPOINT_URL, timeout=10)
        response.raise_for_status()  # Raises exception for 4XX/5XX status codes
        
        launches = response.json()
        logger.info(f"Successfully retrieved {len(launches)} launches")
        return launches
        
    except requests.exceptions.RequestException as e:
        logger.error(f"Failed to retrieve launch data: {str(e)}")
        raise

def aggregate_launches_by_year(launches):
    """Aggregate launches by year"""
    logger.info("Starting aggregation of launches by year")
    
    year_counts = {}
    
    for launch in launches:
        try:
            # Extract year from launch date (UTC)
            launch_date = launch['launch_date_utc']
            year = datetime.strptime(launch_date, '%Y-%m-%dT%H:%M:%S.%fZ').year
            
            # Increment count for this year
            year_counts[year] = year_counts.get(year, 0) + 1
            
        except (KeyError, ValueError) as e:
            logger.warning(f"Skipping launch with invalid date format: {launch.get('flight_number', 'unknown')}")
    
    logger.info(f"Aggregation complete. Found {len(year_counts)} distinct years")
    return year_counts

def upload_results(data):
    """Upload aggregated data to httpbin"""
    logger.info("Starting upload to httpbin")
    
    try:
        # Serialize data to JSON
        json_data = json.dumps(data)
        
        # Make POST request
        response = requests.post(
            HTTPBIN_POST_URL,
            data=json_data,
            headers={'Content-Type': 'application/json'},
            timeout=10
        )
        response.raise_for_status()
        
        logger.info("Upload to httpbin successful")
        return True
        
    except requests.exceptions.RequestException as e:
        logger.error(f"Failed to upload to httpbin: {str(e)}")
        return False

def main():
    """Main execution pipeline"""
    # Start timer for total execution time
    pipeline_start_time = time.time()
    
    try:
        # Measure data retrieval and aggregation time
        start_time = time.time()
        
        # Step 1: Retrieve launch data
        launches = retrieve_launch_data()
        
        # Step 2: Aggregate launches by year
        year_counts = aggregate_launches_by_year(launches)
        
        # Print the aggregated results
        print("\nNumber of launches by year:")
        for year, count in sorted(year_counts.items()):
            print(f"{year}: {count} launches")
        
        # Log execution time for data retrieval + aggregation
        execution_time = time.time() - start_time
        print(f"\nData retrieval and aggregation completed in {execution_time:.2f} seconds")
        
        # Step 3: Upload results
        upload_success = upload_results(year_counts)
        
        # Log total pipeline execution time
        total_time = time.time() - pipeline_start_time
        print(f"\nTotal pipeline execution time: {total_time:.2f} seconds")
        
        return year_counts
        
    except Exception as e:
        logger.error(f"Pipeline failed: {str(e)}")
        raise

if __name__ == "__main__":
    main()